# 5. Model

## 5.0 Read Data

In [1]:
# check xgboost version
import xgboost
# print(xgboost.__version__)
# evaluate an xgboost regression model on the housing dataset
from numpy import absolute
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('xtrain.csv',index_col=0)
test = pd.read_csv('xtest.csv', index_col=0)

In [3]:
test.head()

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_IR1,x2_IR2,x2_Reg,...,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,4.382027,9.360655,0.082920,0.097693,0.115582,0.0,0.144354,0.0,0.250000
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,4.394449,9.565704,0.163536,0.000000,0.173801,0.0,0.217512,0.0,0.333333
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,4.304065,9.534595,0.140149,0.000000,0.058647,0.0,0.151882,0.0,0.333333
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,4.356709,9.208138,0.106662,0.000000,0.138699,0.0,0.151555,0.0,0.416667
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,3.761200,8.518193,0.046598,0.000000,0.435360,0.0,0.209493,0.0,0.250000


In [4]:
# data.dropna(axis= 0, inplace=True)
# data.fillna(0, inplace=True)
target = data['SalePrice']
data.drop('SalePrice',inplace=True, axis=1)
# data.drop('LotFrontage', inplace=True, axis=1)
# data.replace(np.inf, 0, inplace=True)

In [5]:
target = np.log(target)

In [6]:
pd.set_option('display.max_columns', None)
data.head()

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_IR1,x2_IR2,x2_Reg,x2_rare,x3_AllPub,x3_rare,x4_Corner,x4_CulDSac,x4_FR2,x4_Inside,x4_rare,x5_Blmngtn,x5_BrDale,x5_BrkSide,x5_ClearCr,x5_CollgCr,x5_Crawfor,x5_Edwards,x5_Gilbert,x5_IDOTRR,x5_MeadowV,x5_Mitchel,x5_NAmes,x5_NWAmes,x5_NoRidge,x5_NridgHt,x5_OldTown,x5_SWISU,x5_Sawyer,x5_SawyerW,x5_Somerst,x5_StoneBr,x5_Timber,x5_rare,x6_Gable,x6_Hip,x6_rare,x7_CompShg,x7_rare,x8_AsbShng,x8_BrkFace,x8_CemntBd,x8_HdBoard,x8_MetalSd,x8_Plywood,x8_Stucco,x8_VinylSd,x8_Wd Sdng,x8_WdShing,x8_rare,x9_AsbShng,x9_BrkFace,x9_CmentBd,x9_HdBoard,x9_MetalSd,x9_Plywood,x9_Stucco,x9_VinylSd,x9_Wd Sdng,x9_Wd Shng,x9_rare,x10_BrkCmn,x10_BrkFace,x10_None,x10_Stone,x10_rare,x11_BrkTil,x11_CBlock,x11_PConc,x11_Slab,x11_rare,x12_GasA,x12_GasW,x12_rare,x13_N,x13_Y,x14_FuseA,x14_FuseF,x14_SBrkr,x14_rare,x15_Attchd,x15_Basment,x15_BuiltIn,x15_Detchd,x15_No,x15_rare,x16_COD,x16_New,x16_WD,x16_rare,x17_Abnorml,x17_Family,x17_Normal,x17_Partial,x17_rare,x18_Bnk,x18_HLS,x18_Low,x18_Lvl,x19_Gtl,x19_Mod,x19_rare,x20_Artery,x20_Feedr,x20_Norm,x20_PosN,x20_RRAn,x20_rare,x21_Norm,x21_rare,x22_1Fam,x22_2fmCon,x22_Duplex,x22_Twnhs,x22_TwnhsE,x23_1.5Fin,x23_1Story,x23_2Story,x23_SFoyer,x23_SLvl,x23_rare,x24_0,x24_3,x24_4,x24_5,x24_6,x24_7,x24_8,x24_9,x24_10,x25_0,x25_3,x25_4,x25_5,x25_6,x25_7,x25_8,x25_9,x26_Ex,x26_Gd,x26_TA,x26_rare,x27_Fa,x27_Gd,x27_TA,x27_rare,x28_Ex,x28_Fa,x28_Gd,x28_No,x28_TA,x29_Fa,x29_Gd,x29_No,x29_TA,x29_rare,x30_Av,x30_Gd,x30_Mn,x30_No,x31_ALQ,x31_BLQ,x31_GLQ,x31_LwQ,x31_No,x31_Rec,x31_Unf,x32_Ex,x32_Fa,x32_Gd,x32_TA,x32_rare,x33_Fin,x33_No,x33_RFn,x33_Unf,x34_Fa,x34_No,x34_TA,x34_rare,x35_Fa,x35_No,x35_TA,x35_rare,x36_ALQ,x36_BLQ,x36_LwQ,x36_No,x36_Rec,x36_Unf,x36_rare,x37_Ex,x37_Fa,x37_Gd,x37_TA,x38_N,x38_P,x38_Y,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.12250,6.752270,0.413559,0.0,7.444249,0.666667,0.5,0.375,0.333333,0.000000,0.002488,0.386460,0.50,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.090909,0.50,4.174387,9.041922,0.125089,0.0,0.064212,0.333333,0.140098,0.0,0.500000
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.00000,7.140453,0.000000

In [7]:
data.shape

(1460, 235)

In [8]:
data.tail()

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_IR1,x2_IR2,x2_Reg,x2_rare,x3_AllPub,x3_rare,x4_Corner,x4_CulDSac,x4_FR2,x4_Inside,x4_rare,x5_Blmngtn,x5_BrDale,x5_BrkSide,x5_ClearCr,x5_CollgCr,x5_Crawfor,x5_Edwards,x5_Gilbert,x5_IDOTRR,x5_MeadowV,x5_Mitchel,x5_NAmes,x5_NWAmes,x5_NoRidge,x5_NridgHt,x5_OldTown,x5_SWISU,x5_Sawyer,x5_SawyerW,x5_Somerst,x5_StoneBr,x5_Timber,x5_rare,x6_Gable,x6_Hip,x6_rare,x7_CompShg,x7_rare,x8_AsbShng,x8_BrkFace,x8_CemntBd,x8_HdBoard,x8_MetalSd,x8_Plywood,x8_Stucco,x8_VinylSd,x8_Wd Sdng,x8_WdShing,x8_rare,x9_AsbShng,x9_BrkFace,x9_CmentBd,x9_HdBoard,x9_MetalSd,x9_Plywood,x9_Stucco,x9_VinylSd,x9_Wd Sdng,x9_Wd Shng,x9_rare,x10_BrkCmn,x10_BrkFace,x10_None,x10_Stone,x10_rare,x11_BrkTil,x11_CBlock,x11_PConc,x11_Slab,x11_rare,x12_GasA,x12_GasW,x12_rare,x13_N,x13_Y,x14_FuseA,x14_FuseF,x14_SBrkr,x14_rare,x15_Attchd,x15_Basment,x15_BuiltIn,x15_Detchd,x15_No,x15_rare,x16_COD,x16_New,x16_WD,x16_rare,x17_Abnorml,x17_Family,x17_Normal,x17_Partial,x17_rare,x18_Bnk,x18_HLS,x18_Low,x18_Lvl,x19_Gtl,x19_Mod,x19_rare,x20_Artery,x20_Feedr,x20_Norm,x20_PosN,x20_RRAn,x20_rare,x21_Norm,x21_rare,x22_1Fam,x22_2fmCon,x22_Duplex,x22_Twnhs,x22_TwnhsE,x23_1.5Fin,x23_1Story,x23_2Story,x23_SFoyer,x23_SLvl,x23_rare,x24_0,x24_3,x24_4,x24_5,x24_6,x24_7,x24_8,x24_9,x24_10,x25_0,x25_3,x25_4,x25_5,x25_6,x25_7,x25_8,x25_9,x26_Ex,x26_Gd,x26_TA,x26_rare,x27_Fa,x27_Gd,x27_TA,x27_rare,x28_Ex,x28_Fa,x28_Gd,x28_No,x28_TA,x29_Fa,x29_Gd,x29_No,x29_TA,x29_rare,x30_Av,x30_Gd,x30_Mn,x30_No,x31_ALQ,x31_BLQ,x31_GLQ,x31_LwQ,x31_No,x31_Rec,x31_Unf,x32_Ex,x32_Fa,x32_Gd,x32_TA,x32_rare,x33_Fin,x33_No,x33_RFn,x33_Unf,x34_Fa,x34_No,x34_TA,x34_rare,x35_Fa,x35_No,x35_TA,x35_rare,x36_ALQ,x36_BLQ,x36_LwQ,x36_No,x36_Rec,x36_Unf,x36_rare,x37_Ex,x37_Fa,x37_Gd,x37_TA,x38_N,x38_P,x38_Y,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
1455,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,6.859615,0.336077,0.0,7.406711,0.666667,0.5,0.375,0.333333,0.333333,0.003980,0.324401,0.50,0.000000,0.073126,0.000000,0.0,0.0,0.0,0.636364,0.25,4.127134,8.976768,0.000000,0.000000,0.407962,0.000000,0.155974,0.0,0.416667
1456,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.074375,7.63

## 5.1 Train Test Split

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(data, target, test_size=0.4)

## 5.2 SVM Regressor

In [13]:
from sklearn.svm import SVR

In [14]:
# train the model on train set
model = SVR()
model.fit(xtrain, ytrain)
  
# print prediction results
ypred = model.predict(xtest)
errors = mean_squared_error(ytest, ypred)
print(errors)

0.018393891349458378


In [19]:
# defining parameter range
param_grid = {'C': [0.0001, 0.001, 0.01,0.1, 1 , 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['poly', 'rbf', 'sigmoid']}
svr = SVR()
kfold = KFold(n_splits=10)
model = RandomizedSearchCV(svr, param_grid, refit = True, verbose = 3, cv = kfold, scoring= 'neg_mean_squared_error', n_iter=50)

model.fit(xtrain,ytrain)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV 1/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 2/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 3/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 4/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 5/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 6/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 7/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 8/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 9/10] END ................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 10/10] END ...............C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV 1/10] END ...........C=0.01, gamma=0.001, kernel=sigmoid; total time=   0.1s
[CV 2/10] END ...........C=0.01, gamma=0.001, 

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
                   estimator=SVR(), n_iter=50,
                   param_distributions={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10,
                                              100],
                                        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                                        'kernel': ['poly', 'rbf', 'sigmoid']},
                   scoring='neg_mean_squared_error', verbose=3)

In [20]:
# print best parameter after tuning
print(model.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(model.best_estimator_)

{'kernel': 'sigmoid', 'gamma': 0.001, 'C': 100}
SVR(C=100, gamma=0.001, kernel='sigmoid')


In [21]:
# train the model on train set
model = SVR(C=100, gamma=0.001, kernel='sigmoid')
model.fit(xtrain, ytrain)

# print prediction results
ypred = model.predict(xtest)
errors = mean_squared_error(ytest, ypred)
print(errors)

0.016991532985958117


## 5.3 XGBoost

### 5.3.1 Baseline Model

In [25]:
# create a baseline xgboost regression model
model = XGBRegressor()
model.fit(xtrain, ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [26]:
ypred = model.predict(xtest)
errors = mean_squared_error(ytest, ypred)
print(errors)

0.02351532561848901


### 5.3.2 Hyperparameter tuning

In [28]:
# Model Parameters

parameters = {
        'learning_rate': [0.01, 0.1, 1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7, 0.9],
        'colsample_bytree': [0.5, 0.7, 0.9],
        'n_estimators' : [100, 200, 500, 800, 1000],
        'objective': ['reg:squarederror']
    }

In [31]:
kfold = KFold(n_splits=10)

# model = XGBRegressor()

model = RandomizedSearchCV(XGBRegressor(), parameters, refit = True, verbose = 3, cv = kfold, scoring= 'neg_mean_squared_error', n_iter=50)

In [32]:
model.fit(xtrain, ytrain)

, objective=reg:squarederror, subsample=0.9; total time=   0.6s
[CV 4/10] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, min_child_weight=1, n_estimators=100, objective=reg:squarederror, subsample=0.9; total time=   0.5s
[CV 5/10] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, min_child_weight=1, n_estimators=100, objective=reg:squarederror, subsample=0.9; total time=   0.6s
[CV 6/10] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, min_child_weight=1, n_estimators=100, objective=reg:squarederror, subsample=0.9; total time=   0.5s
[CV 7/10] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, min_child_weight=1, n_estimators=100, objective=reg:squarederror, subsample=0.9; total time=   0.8s
[CV 8/10] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, min_child_weight=1, n_estimators=100, objective=reg:squarederror, subsample=0.9; total time=   0.4s
[CV 9/10] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, min_child_weight=1, n_estima

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing...
                                          scale_pos_weight=None, subsample=None,
                                          tree_method=None,
                                          validate_parameters=None,
                                          verbosity=None),
       

In [33]:
# print best parameter after tuning
print(model.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(model.best_estimator_)

{'subsample': 0.5, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)


In [35]:
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

model.fit(xtrain, ytrain)

ypred = model.predict(xtest)
errors = mean_squared_error(ytest, ypred)
print(errors)

0.017831009435021628


In [16]:
indx = pd.read_csv('test.csv')

In [17]:
submission = pd.DataFrame({'Id':indx['Id'],'SalePrice':np.exp(pred)})
submission.SalePrice = submission.SalePrice.apply(np.int64)
submission.to_csv('results.csv', index=False)

## 5.4 AdaBoost Regressor

In [25]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=0, n_estimators=300, learning_rate=0.1)
regr.fit(xtrain,ytrain)

AdaBoostRegressor(learning_rate=0.1, n_estimators=300, random_state=0)

In [26]:
ypred = regr.predict(xtest)

In [27]:
from sklearn.metrics import mean_squared_error
errors = mean_squared_error(ytest, ypred)

In [28]:
errors

0.00021823905467801074

In [ ]:
loss